# **Scraping Komentar YouTube Terkait Bencana Banjir pada Channel Ferry Irwandi**

Program ini digunakan untuk Klasifikasi Teks dan mengambil (scraping) komentar publik dari
video terkait bencana banjir pada channel YouTube Ferry Irwandi
menggunakan YouTube Data API v3.

In [ ]:
#@title Menghubungkan Google Colab dengan Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Install Dependencies

!pip install google-api-python-client

In [ ]:
#@title Menyiapkan API Key Yang Dimiliki Oleh Google

def_api_key = 'YOUR_API_KEY_HERE'  # Ganti dengan API Key Anda

In [ ]:
#@title Menambahkan Library

from googleapiclient.discovery import build
import pandas as pd
import time

In [ ]:
#@title Ambil Playlist Tersembunyi

# Ganti dengan Channel ID (bukan username)
channel_id = 'UCC_OYI6VZtuEZuq49Ht-cQQ'

# Inisialisasi client
youtube = build('youtube', 'v3', developerKey=def_api_key)

def get_uploads_playlist_id(channel_id):
    response = youtube.channels().list(
        part='contentDetails',
        id=channel_id
    ).execute()

    if 'items' not in response or not response['items']:
        print("Channel tidak ditemukan.")
        return None

    uploads_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    return uploads_id

# Jalankan
uploads_playlist_id = get_uploads_playlist_id(channel_id)
print(f"Playlist uploads ID: {uploads_playlist_id}")

Playlist uploads ID: UUC_OYI6VZtuEZuq49Ht-cQQ


In [ ]:
#@title Masukkan Playlist ID ke Variabel

# Playlist tersembunyi
def_playlist_id = uploads_playlist_id

In [ ]:
#@title Masukkan Data Video Ke DataFrame

# Inisialisasi client
youtube = build('youtube', 'v3', developerKey=def_api_key)

def get_videos_from_playlist(playlist_id):
    video_ids = []
    next_page_token = None

    while True:
        res = youtube.playlistItems().list(
            part='contentDetails',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        for item in res['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = res.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(0.1)

    return video_ids

def get_video_details(video_ids):
    video_data = []

    for i in range(0, len(video_ids), 50):
        batch_ids = video_ids[i:i+50]
        res = youtube.videos().list(
            part='snippet,statistics',
            id=','.join(batch_ids)
        ).execute()

        for item in res['items']:
            data = {
                'videoId': item['id'],
                'title': item['snippet']['title'],
                'publishedAt': item['snippet']['publishedAt'],
                'viewCount': int(item['statistics'].get('viewCount', 0)),
                'commentCount': int(item['statistics'].get('commentCount', 0))
            }
            video_data.append(data)

    return video_data

# Ambil semua video ID dari playlist
video_ids = get_videos_from_playlist(def_playlist_id)

# Ambil detail dari video-video tersebut
video_details = get_video_details(video_ids)

# Simpan ke DataFrame
df = pd.DataFrame(video_details)

# Tampilkan 5 baris pertama
print(df.head())

# Simpan ke file CSV (opsional)
df.to_csv("video_playlist.csv", index=False)

       videoId                                              title  \
0  PC3KLjRgRbM  Menembus Banjir Langkat dan Pelosok Tamiang ya...   
1  gHWF3V0w9bI  Mengantarkan Bantuan Dari Masyarakat Indonesia...   
2  r5Ag6hWlIWM     10,3 Miliar Dari Warga Untuk Warga di Sumatera   
3  248xW_-p9HM             8,3 Miliar Dalam 1 Hari Untuk Sumatra!   
4  rFZQQx_Wdhg  Sudah 8 MILIAR RUPIAH ! Untuk Sumatera! live 1...   

            publishedAt  viewCount  commentCount  
0  2025-12-06T03:35:13Z     753679          5126  
1  2025-12-05T03:49:29Z     631411          3443  
2  2025-12-02T13:34:55Z     313369          2208  
3  2025-12-02T03:51:21Z     121133           523  
4  2025-12-02T03:42:35Z     622528           294  


Setelah memasukkan data video ke data frame dan menghasilkan file "video_playlist.csv" langkah selanjutnya adalah download file tersebut dan edit isi file tersebut dengan menghapus video yang tidak akan di scraping. setelah itu upload file tersebut ke google drive.



In [ ]:
#@title Ambil Komentar dari Semua Video

youtube = build('youtube', 'v3', developerKey=def_api_key)

# id pemillik
def_id_pemilik = "@ferryirwandi"
# tentukan jumlah video teratas
def_jumlah_video = 7 #cek dulu di channel nya ferry irwandi ada berapa konten yang membahas banjir sumatra

# Baca CSV video_playlist.csv
df_videos = pd.read_csv('/content/drive/MyDrive/video_playlist.csv')

# Urutkan berdasarkan commentCount descending dan ambil video teratas
df_videos_sorted = df_videos.sort_values(by='commentCount', ascending=False).head(def_jumlah_video)

# Buat dict videoId ke title dari data yang sudah disortir
video_title_map = dict(zip(df_videos_sorted['videoId'], df_videos_sorted['title']))

# Ambil videoId list yang sudah disortir dan dibatasi
video_ids = df_videos_sorted['videoId'].tolist()

def get_comments(video_id):
    comments = []
    request = youtube.commentThreads().list(
        part='snippet,replies',
        videoId=video_id,
        maxResults=100,
        textFormat='plainText'
    )

    while request:
        response = request.execute()
        for item in response['items']:
            top_comment = item['snippet']['topLevelComment']['snippet']
            if (top_comment['authorDisplayName'] == def_id_pemilik and def_id_pemilik != ''):
                continue
            comments.append({
                'author': top_comment['authorDisplayName'],
                'comment': top_comment['textDisplay'],
                'likeCount': top_comment['likeCount'],
                'isReply': False
            })

            if 'replies' in item:
                for reply in item['replies']['comments']:
                    reply_snippet = reply['snippet']
                    if (reply_snippet['authorDisplayName'] == def_id_pemilik and def_id_pemilik != ''):
                        continue
                    comments.append({
                        'author': reply_snippet['authorDisplayName'],
                        'comment': reply_snippet['textDisplay'],
                        'likeCount': reply_snippet['likeCount'],
                        'isReply': True
                    })

        request = youtube.commentThreads().list_next(request, response)
        time.sleep(0.1)

    return comments

def get_video_title(video_id):
    return video_title_map.get(video_id, 'Unknown Title')

all_comment_data = []

batch_size = 10
for i in range(0, len(video_ids), batch_size):
    batch = video_ids[i:i+batch_size]
    print(f"Processing batch videos {i+1} to {i+len(batch)}")

    for video_id in batch:
        print(f"  Mengambil komentar video {video_id}")
        title = get_video_title(video_id)
        comments = get_comments(video_id)

        for c in comments:
            all_comment_data.append({
                'videoId': video_id,
                'title': title,
                'author': c['author'],
                'comment': c['comment'],
                'likeCount': c['likeCount'],
                'isReply': c['isReply']
            })

    time.sleep(10)

df_comments = pd.DataFrame(all_comment_data)
print(df_comments.head())
df_comments.to_csv('video_comments.csv', index=False)

Processing batch videos 1 to 7
  Mengambil komentar video 4loaVrhp75Q
  Mengambil komentar video PC3KLjRgRbM
  Mengambil komentar video gHWF3V0w9bI
  Mengambil komentar video r5Ag6hWlIWM
  Mengambil komentar video 248xW_-p9HM
  Mengambil komentar video YxxZkjACPR8
  Mengambil komentar video rFZQQx_Wdhg
       videoId                                              title  \
0  4loaVrhp75Q  Mengapa Bencana Sumatera Harus Berstatus Benca...   
1  4loaVrhp75Q  Mengapa Bencana Sumatera Harus Berstatus Benca...   
2  4loaVrhp75Q  Mengapa Bencana Sumatera Harus Berstatus Benca...   
3  4loaVrhp75Q  Mengapa Bencana Sumatera Harus Berstatus Benca...   
4  4loaVrhp75Q  Mengapa Bencana Sumatera Harus Berstatus Benca...   

                   author                                            comment  \
0        @bayupratama3429  Lakuin Apa bang? \nKan dia yg Punya Lahan Kons...   
1           @Cutkiabelgia  Aceh tamiang parah pisan.,\nSampek ancur lebur...   
2        @Memory.H.Inside  10 thn menjaba

In [ ]:
#@title Cek Jumlah Data Yang Didapatkan

import pandas as pd
import os

# Nama file hasil scraping
filename = 'video_comments.csv'

# Load data
df_comments = pd.read_csv(filename)

# Tampilkan jumlah data
print("\nJumlah total komentar yang berhasil di-scrap:")
print(len(df_comments))


Jumlah total komentar yang berhasil di-scrap:
15261


In [ ]:
#@title Menampilkan Semua Data

import pandas as pd

# Load data CSV
df_comments = pd.read_csv('video_comments.csv')

# Menampilkan seluruh data
df_comments

,videoId,title,author,comment,likeCount,isReply
0,4loaVrhp75Q,Mengapa Bencana Sumatera Harus Berstatus Benca...,@bayupratama3429,Lakuin Apa bang? \nKan dia yg Punya Lahan Kons...,0,False
1,4loaVrhp75Q,Mengapa Bencana Sumatera Harus Berstatus Benca...,@Cutkiabelgia,"Aceh tamiang parah pisan.,\nSampek ancur lebur...",0,False
2,4loaVrhp75Q,Mengapa Bencana Sumatera Harus Berstatus Benca...,@Memory.H.Inside,10 thn menjabat efek nya besar banget bagi neg...,0,False
3,4loaVrhp75Q,Mengapa Bencana Sumatera Harus Berstatus Benca...,@alfianhadipermana5854,Assalamualaikum bang. Salam kenal dr mantan sa...,0,False
4,4loaVrhp75Q,Mengapa Bencana Sumatera Harus Berstatus Benca...,@HarySi,Yang tidur di masjid di gebukin itu orang mana...,0,False
...,...,...,...,...,...,...
15256,rFZQQx_Wdhg,Sudah 8 MILIAR RUPIAH ! Untuk Sumatera! live 1...,@muhammadsahihmo,makasih banyak bang,1,False
15257,rFZQQx_Wdhg,Sudah 8 MILIAR RUPIAH ! Untuk Sumatera! live 1...,@SmkpMahageperantauan-yi8et,Lyar biasa,1,False
15258,rFZQQx_Wdhg,Sudah 8 MILIAR RUPIAH ! Untuk Sumatera! live 1...,@gadisindonesiarebound,halo,0,False
15259,rFZQQx_Wdhg,Sudah 8 MILIAR RUPIAH ! Untuk Sumatera! live 1...,@Versi_Kita,pertama,0,False


In [ ]:
#@title Menstandarisasi teks
def to_lowercase(text):
    if isinstance(text, str):
        return text.lower()
    return text

# Terapkan case folding pada teks
df_comments['lowercase_text'] = df_comments['comment'].apply(to_lowercase)

df_comments[['comment', 'lowercase_text']].head()

,comment,lowercase_text
0,Lakuin Apa bang? \nKan dia yg Punya Lahan Kons...,lakuin apa bang? \nkan dia yg punya lahan kons...
1,"Aceh tamiang parah pisan.,\nSampek ancur lebur...","aceh tamiang parah pisan.,\nsampek ancur lebur..."
2,10 thn menjabat efek nya besar banget bagi neg...,10 thn menjabat efek nya besar banget bagi neg...
3,Assalamualaikum bang. Salam kenal dr mantan sa...,assalamualaikum bang. salam kenal dr mantan sa...
4,Yang tidur di masjid di gebukin itu orang mana...,yang tidur di masjid di gebukin itu orang mana...


In [ ]:
#@title Normalize Text (Hapus Tanda Baca + Angka)

import string  # <-- WAJIB

# Fungsi untuk menghapus tanda baca dan angka
def normalize_text(text):
    if isinstance(text, str):
        text = text.translate(str.maketrans('', '', string.punctuation))  # Hapus tanda baca
        text = ''.join([i for i in text if not i.isdigit()])              # Hapus angka
        return text
    return text

# Terapkan text normalization
df_comments['normalized_text'] = df_comments['lowercase_text'].apply(normalize_text)
df_comments[['lowercase_text', 'normalized_text']].head()

,lowercase_text,normalized_text
0,lakuin apa bang? \nkan dia yg punya lahan kons...,lakuin apa bang \nkan dia yg punya lahan konse...
1,"aceh tamiang parah pisan.,\nsampek ancur lebur...",aceh tamiang parah pisan\nsampek ancur lebur p...
2,10 thn menjabat efek nya besar banget bagi neg...,thn menjabat efek nya besar banget bagi negar...
3,assalamualaikum bang. salam kenal dr mantan sa...,assalamualaikum bang salam kenal dr mantan sat...
4,yang tidur di masjid di gebukin itu orang mana...,yang tidur di masjid di gebukin itu orang mana ya


In [ ]:
#@title Menghapus kata kata umum yang tidak penting

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download resource NLTK kalau belum
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# Mendapatkan daftar stopwords Bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

# Fungsi untuk menghapus stopwords
def remove_stopwords(text):
    if isinstance(text, str):
        words = word_tokenize(text)
        return ' '.join([word for word in words if word not in stop_words])
    return text

# Terapkan stopword removal pada teks
df_comments['no_stopwords'] = df_comments['normalized_text'].apply(remove_stopwords)

# Tampilkan hasil
df_comments[['normalized_text', 'no_stopwords']].head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,normalized_text,no_stopwords
0,lakuin apa bang \nkan dia yg punya lahan konse...,lakuin bang yg lahan konsesi bumi andalas
1,aceh tamiang parah pisan\nsampek ancur lebur p...,aceh tamiang parah pisan sampek ancur lebur po...
2,thn menjabat efek nya besar banget bagi negar...,thn menjabat efek nya banget negarasuarakan di...
3,assalamualaikum bang salam kenal dr mantan sat...,assalamualaikum bang salam kenal dr mantan atap
4,yang tidur di masjid di gebukin itu orang mana ya,tidur masjid gebukin orang ya


#Di tahap selanjutnya harus sabar karena prosesnya cukup lama. Semangat!

In [ ]:
#@title Mengubah kata menjadi bentuk dasarnya

!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Inisialisasi stemmer Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming
def stemming(text):
    if isinstance(text, str):
        return stemmer.stem(text)
    return text

# Terapkan stemming pada teks
df_comments['komentar'] = df_comments['no_stopwords'].apply(stemming)

# Tampilkan hasil
df_comments[['no_stopwords', 'komentar']].head()


,no_stopwords,komentar
0,lakuin bang yg lahan konsesi bumi andalas,lakuin bang yg lahan konsesi bumi andalas
1,aceh tamiang parah pisan sampek ancur lebur po...,aceh tamiang parah pis sampek ancur lebur pora...
2,thn menjabat efek nya banget negarasuarakan di...,thn jabat efek nya banget negarasuarakan suara
3,assalamualaikum bang salam kenal dr mantan atap,assalamualaikum bang salam kenal dr mantan atap
4,tidur masjid gebukin orang ya,tidur masjid gebukin orang ya


In [ ]:
#@title Simpan hasil prepocessing

# Simpan hanya kolom komentar
df_comments[['komentar']].to_csv("processed_text_step2.csv", index=False)

#Pada tahap auto labeling untuk data train harus sabar ya, karena prosesnya cukup lama bisa sampai 30 menit lebih. Semangat!

In [ ]:
#@title Auto Labeling 600 Data (Zero-Shot + Keyword Fix)

!pip install transformers torch

from transformers import pipeline
import re

# Load zero-shot model
classifier = pipeline(
    "zero-shot-classification",
    model="joeddav/xlm-roberta-large-xnli"
)

# Label kategori final (6 kelas)
labels = [
    "kebijakan pemerintah",
    "pembalakan liar",
    "alih fungsi hutan",
    "perubahan iklim",
    "sosial kemanusiaan",
    "komentar biasa"
]

def auto_label(text):
    text = str(text).strip()
    try:
        result = classifier(text, candidate_labels=labels, multi_label=False)
        return result['labels'][0]
    except:
        return "lainnya"  # sementara "lainnya" → akan diperbaiki belakangan


# Keyword fallback mapping
keyword_map = {
    "kebijakan pemerintah": [
        "pemerintah", "presiden", "menteri", "dpr", "aturan", "kebijakan", "perda"
    ],
    "pembalakan liar": [
        "illegal", "tebang", "kayu", "pembalak", "loging", "logging"
    ],
    "alih fungsi hutan": [
        "tambang", "sawit", "bangun", "perumahan", "industri", "proyek"
    ],
    "perubahan iklim": [
        "iklim", "cuaca", "panas", "global", "emisi", "pemanasan"
    ],
    "sosial kemanusiaan": [
        "korban", "warga", "bencana", "bantu", "meninggal", "evakuasi"
    ],
    "komentar biasa": [
        "info", "berita", "lapor", "update", "video", "cek", "sumber"
    ]
}

def reassign_label(row):
    text = row["komentar"].lower()
    label = row["label"]

    # Jika sudah 6 label utama → biarkan
    if label in labels:
        return label

    # Cek keyword untuk ubah "lainnya"
    for new_label, keywords in keyword_map.items():
        for kw in keywords:
            if re.search(rf"\b{kw}\b", text):
                return new_label

    # Tanpa keyword → fallback
    return "komentar biasa"


# Ambil sample 1000 untuk zero-shot labeling
df_sample = df_comments.sample(600, random_state=42).copy()

# Auto-label dengan zero-shot
df_sample["label"] = df_sample["komentar"].apply(auto_label)

# Bersihkan data kosong
df_sample["komentar"] = df_sample["komentar"].astype(str)
df_sample = df_sample[df_sample["komentar"].str.strip() != ""]
df_sample = df_sample.dropna(subset=["komentar"])

# Re-label kategori "lainnya" agar jadi salah satu dari 6 kelas
df_sample["label"] = df_sample.apply(reassign_label, axis=1)

# Simpan hasil akhir hanya 2 kolom
df_sample[["komentar", "label"]].to_csv("labeled_600_clean.csv", index=False)

print("Distribusi label 6 kategori:")
print(df_sample["label"].value_counts())

df_sample[["komentar", "label"]].to_csv("labeled_600_clean.csv", index=False)


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Distribusi label 6 kategori:
label
sosial kemanusiaan      296
pembalakan liar         101
kebijakan pemerintah     99
alih fungsi hutan        56
komentar biasa           25
perubahan iklim          15
Name: count, dtype: int64


In [ ]:
#@title Split Dataset: Train-Test (80% - 20%)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_sample["komentar"],
    df_sample["label"],
    test_size=0.2,
    random_state=42,
    stratify=df_sample["label"]
)

In [ ]:
#@title TF-IDF Vectorization (Feature Extraction)

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
#@title Train Model SVM + Evaluation

from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

model = LinearSVC()
model.fit(X_train_tfidf, y_train)

pred = model.predict(X_test_tfidf)
print(classification_report(y_test, pred))

                      precision    recall  f1-score   support

   alih fungsi hutan       0.75      0.27      0.40        11
kebijakan pemerintah       0.57      0.60      0.59        20
      komentar biasa       0.00      0.00      0.00         5
     pembalakan liar       0.43      0.15      0.22        20
     perubahan iklim       0.00      0.00      0.00         3
  sosial kemanusiaan       0.63      0.90      0.74        60

            accuracy                           0.61       119
           macro avg       0.40      0.32      0.32       119
        weighted avg       0.55      0.61      0.55       119



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
#@title Buang baris yang kolom komentar kosong atau NaN

df_comments = df_comments.dropna(subset=["komentar"])
df_comments = df_comments[df_comments["komentar"].str.strip() != ""]

df_comments.reset_index(drop=True, inplace=True)

print("Sisa data setelah bersih:", len(df_comments))


Sisa data setelah bersih: 14908


In [ ]:
#@title Transform Semua Data ke TF-IDF

X_all_tfidf = vectorizer.transform(df_comments["komentar"])

In [ ]:
#@title Prediksi Label Seluruh Data + Simpan ke CSV

X_all_tfidf = vectorizer.transform(df_comments["komentar"])
df_comments["predicted_label"] = model.predict(X_all_tfidf)

df_comments.to_csv("data_final_yt.csv", index=False)
print("Selesai! File disimpan --> data_final_yt.csv")

Selesai! File disimpan --> data_final_yt.csv


In [ ]:
#@title Cek Total Data & Tampilkan Contoh Hasil Label

print("Total data:", len(df_comments))
df_comments[["predicted_label", "komentar"]].head()

Total data: 14908


,predicted_label,komentar
0,sosial kemanusiaan,lakuin bang yg lahan konsesi bumi andalas
1,sosial kemanusiaan,aceh tamiang parah pis sampek ancur lebur pora...
2,pembalakan liar,thn jabat efek nya banget negarasuarakan suara
3,sosial kemanusiaan,assalamualaikum bang salam kenal dr mantan atap
4,sosial kemanusiaan,tidur masjid gebukin orang ya


In [ ]:
#@title Statistik Distribusi Label Hasil Prediksi

label_count = df_comments["predicted_label"].value_counts()
label_percent = (df_comments["predicted_label"].value_counts(normalize=True) * 100).round(2)

df_label_stats = pd.DataFrame({
    "Jumlah": label_count,
    "Persentase (%)": label_percent
})

print(df_label_stats)

                      Jumlah  Persentase (%)
predicted_label                             
sosial kemanusiaan      9977           66.92
kebijakan pemerintah    2437           16.35
pembalakan liar         1454            9.75
alih fungsi hutan        873            5.86
komentar biasa           125            0.84
perubahan iklim           42            0.28


In [ ]:
#@title Simpan File CSV (Hanya Kolom Teks dan Label)

# Pilih kolom yang ingin disimpan
df_to_save = df_comments[["predicted_label", "komentar"]]

# Cek lokasi file hasil sebelumnya
print("File hasil saat ini:", "data_final_yt.csv")

# Tentukan path tujuan di Google Drive
save_path = "/content/drive/MyDrive/SCRAPING DATA/Output/Comment_YT_Banjir.csv"

# Simpan ke Drive
df_to_save.to_csv(save_path, index=False)

print("Berhasil disimpan ke:", save_path)


File hasil saat ini: data_final_yt.csv
Berhasil disimpan ke: /content/drive/MyDrive/DRAFT PROJECT AKHIR PBA/DATASET TEXT PROCESSING/Comment_YT_Banjir.csv
